# Final Project

## Data Exploration

In [4]:
from xml.dom.minidom import parse
from math import log
import xml
import os
import sys
import csv 
import json
import re
import requests
import math
# from tqdm import tqdm_notebook
import xml.dom.minidom
from collections import Counter
data_path = r"C:\Programming\Python\Programs_NLP\wikihowcom-20141208-current.xml"
if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")


In [5]:
def parse_data(file):
    file_dict = {}
    # Open XML document using minidom parser
    DOMTree = xml.dom.minidom.parse(file)
    mediawiki = DOMTree.documentElement
    #if mediawiki.hasAttribute("xmlns"):
        #print("Root element : %s" % mediawiki.getAttribute("xmlns"))
    # Get all the pages on file
    pages = mediawiki.getElementsByTagName("page")
    # Get high level namespace information.
    for page in pages:
        page_dom = page.getElementsByTagName('text')[0]
        dom_text = page_dom.childNodes[0].data
        if str(dom_text[0]) == '#':
            pass
        else:
            id_ = page.getElementsByTagName('id')[0]
            id_ = id_.childNodes[0].data
            title = page.getElementsByTagName('title')[0]
            title = title.childNodes[0].data
            text = page.getElementsByTagName('text')[0]
            text = text.childNodes[0].data
            file_dict[int(id_)] = [title, text]
    return file_dict


#data_dict = parse_data(data_path)

def structure(page_id, file_dict):
    file_ = file_dict[page_id][1]
    lines = file_.split("\n")
    lines_ = []
    for line in lines:
        if line != "":
            lines_.append(line)
    intro = lines_[0]
    try:
        if "Category" in lines_[1]:
            cat = lines_[1]
            if ":" in cat:
                cat = cat.split(":")[1][:-2]
        else:
            cat = None
    except IndexError:
        cat = None
    try:
        if "Category" in lines[2] and cat is not None:
            cat = cat + "," + re.sub('[^A-Za-z0-9\s\.|\:\*]+', '', lines[2]).split(":")[1]
            process = lines_[3:]
        else:
            process = lines_[2:]
    except IndexError:
        process = None

    return intro, cat, process

def merge(data_list):
    try:
        data_list = " ".join(data_list)
        data_list = data_list.split(" ==")
        data_list = list(zip(*[iter(data_list)] * 2))
    except TypeError:
        data_list = []
    return data_list

def construct_dict(data_list):
    textdict = {}
    steps_dict = {}
    #construct step dictt
    try:
        if data_list[0][1] != "":
            title = re.sub(r'([^\:\w])+', '', data_list[0][0]) 
            #print(title)
            steps = data_list[0][1].split(" #")
            steps = [st for st in steps if st != ""]
            step_index = ["step" + str(number + 1) for number in range(len(steps))]
            step_data = [ste.split("<br><br>") for ste in steps]
            step_new =  []
            for step in step_data:
                step_new.append(step[0])  
            step_data = step_new
            steps_dict = dict(zip(step_index, step_data))
            #return steps_dict
            textdict["steps"] = steps_dict
        subsections = []
        for sect in data_list[1:]:
            if sect == "":
                sect.pop(0)
                #print(sect)
            title = sect[0]
            title = re.sub(r'([^\:\w_])+', '', title)
            title  = "".join([t for t in title if t != " "])
            #print(sect[1:])
            content = sect[1]
            if content[1] == "*":
                subs = content.split("*")
                subs = [s for s in subs if s != " "]
                subs = [re.sub('[^A-Za-z0-9\s\.|\:\*]+', '', string) for string in subs]
                textdict[title] = subs
        return textdict
    except IndexError:
        return {}


def parse_(page_id, data_dict):
    structured = structure(page_id, data_dict)
    final_dict = construct_dict(merge(structured[2]))
    final_dict["title"] = data_dict[page_id][0]
    final_dict["introduction"] = structured[0]
    final_dict["category"] = structured[1]
    final_dict["id"] = page_id
    #parsed = json.loads(json.dumps(final_dict, ensure_ascii=False, sort_keys=True))
    #print(json.dumps(parsed, indent=4, sort_keys=True))
    return final_dict

## Parse Data (this will tax your memory)

In [6]:
data_dict = parse_data(data_path)

In [7]:
cat_list = []
for val in data_dict.values():
    cat = re.search(r'(?<=\[\[category:)[^\s\\\]]*',val[1],flags=re.I)
    if cat is not None:
        cat_list.append(cat.group(0))

In [8]:
cat_counter = Counter(cat_list)

In [9]:
class Post(object):
    
    @staticmethod
    def decode(regex, sep=None):
        if regex is None:
            return None
        else:
            if sep is None:
                return regex.group(0)
            else:
                return [w.strip() for w in regex.group(0).split(sep) if w != '']
    
    def __init__(self,entry):
        section_regex = lambda section: r'(?<= {} \=\=)[\s\S]+?(?=\=\= |\Z)'.format(section)
        
        self.title = entry[0]
        self.text = entry[1]
        cat = re.search(r'(?<=\[\[category:)[^\s\\\]]*',val[1],flags=re.I)
        self.category =  cat.group(0) if cat is not None else None
        self.intro = Post.decode(re.search(r'[\s\S]+(?=\[\[category)',self.text,re.M|re.I))
        self.steps = Post.decode(re.search(section_regex('steps'),self.text,re.M | re.I), sep='\n#')
        self.tips = Post.decode(re.search(section_regex('tips'),self.text,re.M | re.I), sep='\n*')
        self.warnings = Post.decode(re.search(section_regex('warnings'),self.text,re.M | re.I), sep='\n*')
        self.tools = Post.decode(re.search(section_regex('you\'ll need'),self.text,re.M | re.I), sep='\n*')
        self.related_posts = Post.decode(re.search(section_regex('related wikihows'),self.text,re.M | re.I), sep='\n*')

In [ ]:
post_object_list = []
for val in data_dict.values():
    post_object_list.append(Post(val))

In [311]:
import pickle
pickle.dump( post_object_list, open( "posts.p", "wb" ) )